<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/MaestroBreakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib openpyxl TvDatafeed

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının zaman dilimi sayısını seçmesini sağlayan fonksiyon
def get_interval_count_choice():
    print("Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?")
    print("1: Tek Zaman Dilimi")
    print("2: İki Zaman Dilimi")

    choice = input("Seçiminizi yapın (1-2): ")

    if choice == '1':
        return 1
    elif choice == '2':
        return 2
    else:
        print("Geçersiz seçim, varsayılan olarak iki zaman dilimi seçildi.")
        return 2

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice(count=2):
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute, 50), #bar uzunluğu
        '2': ('30 Dakika', Interval.in_30_minute, 40),
        '3': ('45 Dakika', Interval.in_45_minute, 35),
        '4': ('1 Saat', Interval.in_1_hour, 30),
        '5': ('2 Saat', Interval.in_2_hour, 25),
        '6': ('4 Saat', Interval.in_4_hour, 20),
        '7': ('Günlük', Interval.in_daily, 20),
        '8': ('Haftalık', Interval.in_weekly, 10),
        '9': ('Aylık', Interval.in_monthly, 5)
    }

    print("Lütfen zaman dilimi(lerini) seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice_1 = input("Birinci zaman dilimini seçin (1-9): ")
    interval_1 = intervals.get(choice_1, ('4 Saat', Interval.in_4_hour, 20))

    if count == 2:
        choice_2 = input("İkinci zaman dilimini seçin (1-9): ")
        interval_2 = intervals.get(choice_2, ('Günlük', Interval.in_daily, 20))
        return [interval_1, interval_2]

    return [interval_1]

# Breakout stratejisini uygular
def breakout_strategy(data, period):
    df = data.copy()
    df['High_Max'] = df['High'].rolling(window=period).max()
    df['Low_Min'] = df['Low'].rolling(window=period).min()
    df['Breakout_Up'] = df['Close'] > df['High_Max'].shift(1)
    df['Breakout_Down'] = df['Close'] < df['Low_Min'].shift(1)
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Zaman dilimi sayısını seçme
interval_count = get_interval_count_choice()

# Kullanıcının seçimine göre zaman dilimlerini al
intervals = get_interval_choice(interval_count)

df_breakout_up = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Breakout Türü', 'Zaman Dilimi', 'Kırılım Zamanı'])
df_breakout_down = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Breakout Türü', 'Zaman Dilimi', 'Kırılım Zamanı'])

# Seçilen zaman dilimlerine göre analiz yapar
for interval_data in intervals:
    interval_name = interval_data[0]
    interval = interval_data[1]
    period = interval_data[2]  # Zaman dilimine göre dinamik periyot

    for i in range(len(Hisseler)):
        try:
            data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=interval, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()
            breakout_data = breakout_strategy(data, period=period)
            breakout_data['datetime'] = pd.to_datetime(breakout_data['datetime'])
            breakout_data.set_index('datetime', inplace=True)
            last_signal = breakout_data.iloc[-1]
            Breakout_Up = last_signal['Breakout_Up']
            Breakout_Down = last_signal['Breakout_Down']
            Last_Price = last_signal['Close']
            Signal_Time = last_signal.name  # Kırılım zamanı

            if Breakout_Up:
                df_breakout_up.loc[len(df_breakout_up)] = [Hisseler[i], Last_Price, 'Breakout Yukarı', interval_name, Signal_Time]
                print(f"Tamamlandı: {Hisseler[i]} - {interval_name} - Breakout Yukarı - Kırılım Zamanı: {Signal_Time}")
            elif Breakout_Down:
                df_breakout_down.loc[len(df_breakout_down)] = [Hisseler[i], Last_Price, 'Breakout Aşağı', interval_name, Signal_Time]
                print(f"Tamamlandı: {Hisseler[i]} - {interval_name} - Breakout Aşağı - Kırılım Zamanı: {Signal_Time}")
        except Exception as e:
            print(f"Error processing {Hisseler[i]}: {e}")

# Breakout Yukarı ve Aşağı sinyallerini tek bir tabloya ekleyelim
df_all_signals = pd.concat([df_breakout_up, df_breakout_down]).reset_index(drop=True)

# Ekranda tablo olarak gösterme
print("Breakout Yukarı Sinyalleri:")
print(df_breakout_up)
print("\nBreakout Aşağı Sinyalleri:")
print(df_breakout_down)

# Excel dosyasına kaydetme
excel_file = 'breakout_signals.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    df_breakout_up.to_excel(writer, sheet_name='Breakout Yukarı', index=False)
    df_breakout_down.to_excel(writer, sheet_name='Breakout Aşağı', index=False)
    df_all_signals.to_excel(writer, sheet_name='Tüm Sinyaller', index=False)

print(f"\nBreakout yukarı ve aşağı sinyalleri '{excel_file}' dosyasına kaydedildi.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-2afh6_17
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-2afh6_17
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?
1: Tek Zaman Dilimi
2: İki Zaman Dilimi
Seçiminizi yapın (1-2): 2
Lütfen zaman dilimi(lerini) seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Birinci zaman dilimini seçin (1-9): 7
İkinci zaman dilimini seçin (1-9): 5
Tamamlandı: AHGAZ - Günlük - Breakout Yukarı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: ALVES - Günlük - Breakout Aşağı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: BAYRK - Günlük - Breakout Aşağı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: BOSSA - Günlük - Breakout Aşağı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: BRKO - Günlük - Breakout Yukarı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: CEOEM - Günlük - Breakout Yukarı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: CVKMD - Günlük - Breakout Aşağı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: DAPGM - Günlük - Breakout Aşağı - Kırılım Zamanı: 2024-10-04 06:00:00
Tamamlandı: